<img src='img/logo.png'>
<img src='img/title.png'>

This notebook describes how to quantitatively assess models' information value by calculating classification metrics like receiver operating characteristic (ROC) curves and confusion matrices.

# Table of Contents
* [Classification](#Classification)
* [Evaluation metrics and scoring](#Evaluation-metrics-and-scoring)
	* [Metrics for binary classification](#Metrics-for-binary-classification)
		* [Confusion matrices](#Confusion-matrices)
			* [Relation to accuracy](#Relation-to-accuracy)
		* [Precision, recall and f-score](#Precision,-recall-and-f-score)
* [Taking uncertainty into account](#Taking-uncertainty-into-account)
	* [Precision-Recall curves and ROC curves](#Precision-Recall-curves-and-ROC-curves)
		* [ROC and AUC](#ROC-and-AUC)
	* [Multi-class classification](#Multi-class-classification)
* [Summary](#Summary)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams['image.interpolation'] = "none"
np.set_printoptions(precision=3)
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['legend.numpoints'] = 1

import src.mglearn as mglearn

# Classification

Classification is the set of techniques to predict a label, either binary or from a set, based on the features.

Simple classifiers predict the label of a new observation based on proximity to a known observation.

More generalized classifiers attempt to find dividing surfaces within the n-dimesional feature space to decide how to label a new observation.

# Evaluation metrics and scoring

Reading on classification metrics:

http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix

http://scikit-learn.org/stable/modules/model_evaluation.html#classification-report

http://scikit-learn.org/stable/modules/model_evaluation.html#precision-recall-and-f-measures

## Metrics for binary classification

In [ ]:
from sklearn.model_selection import train_test_split

data = pd.read_csv(os.path.join("data", "bank-campaign.csv"))
X = data.drop("target", axis=1).values
y = data.target.values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


In [ ]:
from sklearn.dummy import DummyClassifier
dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
pred_most_frequent = dummy_majority.predict(X_test)
print("predicted labels: %s" % np.unique(pred_most_frequent))
print("score: %f" % dummy_majority.score(X_test, y_test))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train)
pred_tree = tree.predict(X_test)
tree.score(X_test, y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

dummy = DummyClassifier().fit(X_train, y_train)
pred_dummy = dummy.predict(X_test)
print("dummy score: %f" % dummy.score(X_test, y_test))

logreg = LogisticRegression(C=0.1).fit(X_train, y_train)
pred_logreg = logreg.predict(X_test)
print("logreg score: %f" % logreg.score(X_test, y_test))

### Confusion matrices

Confusion matrices display the total amount of true and false positive and negative classifcations.

In [ ]:
mglearn.plots.plot_binary_confusion_matrix()

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, pred_logreg)
print(confusion)

Fractions may be more useful to compare

In [ ]:
n_no = y_test[y_test=='no'].shape[0]
n_yes = y_test[y_test=='yes'].shape[0]

sizes = np.array([n_no, n_yes])

confusion / sizes

In [ ]:
print("Most frequent class:")
print(confusion_matrix(y_test, pred_most_frequent)/sizes)
print("\nDummy model:")
print(confusion_matrix(y_test, pred_dummy)/sizes)
print("\nDecision tree:")
print(confusion_matrix(y_test, pred_tree)/sizes)
print("\nLogistic Regression")
print(confusion_matrix(y_test, pred_logreg)/sizes)

#### Relation to accuracy

\begin{equation}
\text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}
\end{equation}

### Precision, recall and f-score

\begin{equation}
\text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}
\end{equation}

\begin{equation}
\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}
\end{equation}
\begin{equation}
\text{F1} = 2 \cdot \frac{\text{precision} \cdot \text{recall}}{\text{precision} + \text{recall}}
\end{equation}

In [ ]:
from sklearn.metrics import f1_score
print("f1 score most frequent: %.2f" % f1_score(y_test, pred_most_frequent, pos_label="yes"))
print("f1 score dummy: %.2f" % f1_score(y_test, pred_dummy, pos_label="yes"))
print("f1 score tree: %.2f" % f1_score(y_test, pred_tree, pos_label="yes"))
print("f1 score: %.2f" % f1_score(y_test, pred_logreg, pos_label="yes"))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_most_frequent,
                            target_names=["no", "yes"]))

In [ ]:
print(classification_report(y_test, pred_tree,
                            target_names=["no", "yes"]))

In [ ]:
print(classification_report(y_test, pred_logreg,
                            target_names=["no", "yes"]))

# Taking uncertainty into account

In [ ]:
from src.mglearn.datasets import make_blobs 
from sklearn.svm import SVC
X, y = make_blobs(n_samples=(400, 50), centers=2, cluster_std=[7.0, 2],        
                  random_state=22)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
svc = SVC(gamma=.05).fit(X_train, y_train)                                     


In [ ]:
mglearn.plots.plot_decision_threshold()

In [ ]:
print(classification_report(y_test, svc.predict(X_test)))

In [ ]:
y_pred_lower_threshold = svc.decision_function(X_test) > -.8

In [ ]:
print(classification_report(y_test, y_pred_lower_threshold))

## Precision-Recall curves and ROC curves

Curves allow us to view tradeoff between precision and recall or true positive rate versus false positive rate (ROC).

http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

also want to look at:

http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#example-model-selection-plot-roc-crossval-py

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test,
                                                       svc.decision_function(X_test))

In [ ]:
# create a similar dataset as before, but with more samples to get a smoother curve
X, y = make_blobs(n_samples=(4000, 500), centers=2, cluster_std=[7.0, 2], random_state=22)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

svc = SVC(gamma=.05).fit(X_train, y_train)

precision, recall, thresholds = precision_recall_curve(
    y_test, svc.decision_function(X_test))
# find threshold closest to zero:
close_zero = np.argmin(np.abs(thresholds))
plt.figure()
plt.plot(precision[close_zero], recall[close_zero], 'o', markersize=10,
         label="threshold zero", fillstyle="none", c='k', mew=2)

plt.plot(precision, recall, label="precision recall curve")
plt.xlabel("precision")
plt.ylabel("recall")
plt.title("precision_recall_curve");
plt.legend(loc="best");

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=0, max_features=2)
rf.fit(X_train, y_train)

# RandomForestClassifier has predict_proba, but not decision_function
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(
    y_test, rf.predict_proba(X_test)[:, 1])
plt.figure()

plt.plot(precision, recall, label="svc")

plt.plot(precision[close_zero], recall[close_zero], 'o', markersize=10,
         label="threshold zero svc", fillstyle="none", c='k', mew=2)

plt.plot(precision_rf, recall_rf, label="rf")

close_default_rf = np.argmin(np.abs(thresholds_rf - 0.5))
plt.plot(precision_rf[close_default_rf], recall_rf[close_default_rf], '^', markersize=10,
         label="threshold 0.5 rf", fillstyle="none", c='k', mew=2)
plt.xlabel("precision")
plt.ylabel("recall")
plt.legend(loc="best")
plt.title("precision_recall_comparison");

In [ ]:
print("f1_score of random forest: %f" % f1_score(y_test, rf.predict(X_test)))
print("f1_score of svc: %f" % f1_score(y_test, svc.predict(X_test)))

In [ ]:
from sklearn.metrics import average_precision_score
ap_rf = average_precision_score(y_test, rf.predict_proba(X_test)[:, 1])
ap_svc = average_precision_score(y_test, svc.decision_function(X_test))
print("average precision of random forest: %f" % ap_rf)
print("average precision of svc: %f" % ap_svc)

### ROC and AUC

When looking at ROC plots, it is also helpful to calculate AUC, the area under the curve.  A perfect classifier has an AUC of 1.0.

\begin{equation}
\text{FPR} = \frac{\text{FP}}{\text{FP} + \text{TN}}
\end{equation}

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, svc.decision_function(X_test))

plt.figure()

plt.plot(fpr, tpr, label="ROC Curve")
plt.xlabel("FPR")
plt.ylabel("TPR (recall)")
plt.title("roc_curve");
# find threshold closest to zero:
close_zero = np.argmin(np.abs(thresholds))
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize=10,
         label="threshold zero", fillstyle="none", c='k', mew=2)
plt.legend(loc=4);

In [ ]:
from sklearn.metrics import roc_curve
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, rf.predict_proba(X_test)[:, 1])

plt.figure()
plt.plot(fpr, tpr, label="ROC Curve SVC")
plt.plot(fpr_rf, tpr_rf, label="ROC Curve RF")

plt.xlabel("FPR")
plt.ylabel("TPR (recall)")
plt.title("roc_curve_comparison");
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize=10,
         label="threshold zero SVC", fillstyle="none", c='k', mew=2)
close_default_rf = np.argmin(np.abs(thresholds_rf - 0.5))
plt.plot(fpr_rf[close_default_rf], tpr[close_default_rf], '^', markersize=10,
         label="threshold 0.5 RF", fillstyle="none", c='k', mew=2)

plt.legend(loc=4);

In [ ]:
from sklearn.metrics import roc_auc_score
rf_auc = roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1])
svc_auc = roc_auc_score(y_test, svc.decision_function(X_test))
print("AUC for Random Forest: %f" % rf_auc)
print("AUC for SVC: %f" % svc_auc)

In [ ]:
X = data.drop("target", axis=1).values
y = data.target.values
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0, train_size=.1, test_size=.1)

plt.figure()

for gamma in [1, 0.01, 0.001]:
    svc = SVC(gamma=gamma).fit(X_train, y_train)
    accuracy = svc.score(X_test, y_test)
    auc = roc_auc_score(y_test == "yes", svc.decision_function(X_test))
    fpr, tpr, _ = roc_curve(y_test , svc.decision_function(X_test), pos_label="yes")
    print("gamma = %.03f  accuracy = %.02f  AUC = %.02f" % (gamma, accuracy, auc))
    plt.plot(fpr, tpr, label="gamma=%.03f" % gamma, linewidth=4)
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.xlim(-0.01, 1)
plt.ylim(0, 1.02)
plt.legend(loc="best");

## Multi-class classification

In a binary classification problem, the confusion matrix has a shape of `(2, 2)`.  In a multi-class problem, the confusion matrix has a shape `(n_classes, n_classes)` to show the correctly vs incorrectly classified counts for each class versus every other class.

A perfect confusion matrix has positive main diagonal and zeros elsewhere.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits
digits = load_digits()

X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target, random_state=0)
lr = LogisticRegression().fit(X_train, y_train)
pred = lr.predict(X_test)
print("accuracy: %0.3f" % accuracy_score(y_test, pred))
print("confusion matrix:")
print(confusion_matrix(y_test, pred))

In [ ]:
plt.figure()
scores_image = mglearn.tools.heatmap(confusion_matrix(y_test, pred), xlabel='Predicted label', ylabel='True label',
                                     xticklabels=digits.target_names, yticklabels=digits.target_names,
                                     cmap=plt.cm.gray_r, fmt="%d")    

plt.title("Confusion matrix")
plt.gca().invert_yaxis()

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
# This takes a couple minutes
print("micro average f1 score: %f" % f1_score(y_test, pred, average="micro"))
print("macro average f1 score: %f" % f1_score(y_test, pred, average="macro"))

In [ ]:
from sklearn.metrics.scorer import SCORERS
print(sorted(SCORERS.keys()))

# Summary

In this notebook, we reviewed the following topics in preparation for more advanced topics:

 * [Evaluation metrics and scoring](#Evaluation-Metrics-and-scoring)
 * [Metrics for binary classification](#Metrics-for-binary-classification)
 * [Confusion matrices](#Confusion-matrices)
 * [Precision, recall and f-score](#Precision,-recall-and-f-score)
 * [Precision-Recall curves and ROC curves](#Precision-Recall-curves-and-ROC-curves)
 * [ROC and AUC](#Receiver-Operating-Characteristics-%28ROC%29-and-AUC)
 * [Multi-class classification](#Multi-class-classification)

<a href='Evaluation_Metrics_Exercises.ipynb' class='btn btn-primary btn-lg'>Exercises</a>

<img src='img/copyright.png'>